# Whirlwind tour

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/sciris/sciris/HEAD?labpath=docs%2Ftutorials%2Ftut_intro.ipynb) to open an interactive version of this notebook.
    
</div>


Doing science in Python is fun. But it can also be a pain. Sciris cann't make it any more fun, but hopefully it can make it less of a pain. This tutorial showcases some of Sciris' most commonly used features, which are intended to help Python get out of the way of your science. It won't write your code for you like [ChatGPT](https://chat.openai.com), but it does mean you have less code to write.

<p>A Collapsible:</p>
<button type="button" class="collapsible">Open Collapsible</button>
<div class="content">
  <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
</div>

## TBC

TBC

In [2]:
import sciris as sc

sc.findinds([1,2,3],2)

array([1])